# TF Data Preparation

In [2]:
from PetReader import pet_reader
import tensorflow as tf
import tensorflow_addons  as tfa
import transformers
from petreader.labels import *

Reusing dataset pet (C:\Users\janek\.cache\huggingface\datasets\patriziobellan___pet\relations-extraction\1.0.1\38434e2af57af533c400c8975f37e43c08bb77739085a3c026a862b2efb668d2)




 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]



  0%|          | 0/1 [00:00<?, ?it/s]

Reusing dataset pet (C:\Users\janek\.cache\huggingface\datasets\patriziobellan___pet\token-classification\1.0.1\38434e2af57af533c400c8975f37e43c08bb77739085a3c026a862b2efb668d2)




 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]



  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
import argparse

## Create Dataset

In [3]:
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [163]:
print(pet_reader.token_dataset.GetSampleDictWithNerLabels(316))

{'document name': 'doc-5.4', 'sentence-ID': 12, 'tokens': ['If', 'the', 'treasurer', 'accepts', 'the', 'expenses', 'for', 'processing', ',', 'the', 'report', 'moves', 'to', 'an', 'automatic', 'activity', 'that', 'links', 'to', 'a', 'payment', 'system', '.'], 'ner-tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [239]:
WEIGHTS_BERT_TOKENS = 0
WEIGHTS_GATEWAY_LABELS = 1
WEIGHTS_OTHER = 0.2

sample_numbers = pet_reader.token_dataset.GetRandomizedSampleNumbers()
sample_dicts = [pet_reader.token_dataset.GetSampleDictWithNerLabels(sample_number) for sample_number in sample_numbers]
sample_sentences = [sample_dict['tokens'] for sample_dict in sample_dicts]

# 1) transform tokens tags into IDs classification
dataset_tokens = tokenizer(sample_sentences, is_split_into_words=True, padding=True, return_tensors='tf')
max_sentence_length = all_tokens['input_ids'].shape[1]

# 2) transform NER token tags into labels for classification
dataset_labels = []
dataset_sample_weights = []
dataset_word_ids = []
for i, sample_number in enumerate(sample_numbers):
    sample_dict = pet_reader.token_dataset.GetSampleDictWithNerLabels(sample_number)
    # transformer_tokens = tokenizer.convert_ids_to_tokens(tokens['input_ids'][i])
    # tokenize again every single sample to get access to .word_ids()
    tokenization = tokenizer(sample_dict['tokens'], is_split_into_words=True, 
                             padding='max_length', max_length=max_sentence_length, return_tensors='tf')
    sample_tokens = tokenizer.convert_ids_to_tokens(tokenization['input_ids'][0])
    
    sample_labels = []
    sample_sample_weights = []
    # word index necessary, because one token in PET could be splitted into multiple tokens with tokenizer
    # multiple tokens have all the same word_id -> allows retrieval of the same one NER label from PET tokens
    for token, word_index in zip(sample_tokens, tokenization.word_ids()):
        # set special class for special tokens
        if token in ['[CLS]', '[SEP]', '[PAD]']:
            sample_labels.append(0)
            sample_sample_weights.append(WEIGHTS_BERT_TOKENS)
        else:
            
            token_tag = sample_dict['ner-tags'][word_index]
            # XOR
            if token_tag.endswith(XOR_GATEWAY):
                sample_labels.append(2)
                sample_sample_weights.append(1)
            # AND
            elif token_tag.endswith(AND_GATEWAY):
                sample_labels.append(3)
                sample_sample_weights.append(1)
            # Other
            elif token_tag.endswith("O"):
                sample_labels.append(1)
                sample_sample_weights.append(WEIGHTS_OTHER)
            # Activity
            elif token_tag.endswith(ACTIVITY):
                sample_labels.append(4)
                sample_sample_weights.append(WEIGHTS_OTHER)
            # Activity Data
            elif token_tag.endswith(ACTIVITY_DATA):
                sample_labels.append(5)
                sample_sample_weights.append(WEIGHTS_OTHER)
            # Actor
            elif token_tag.endswith(ACTOR):
                sample_labels.append(6)
                sample_sample_weights.append(WEIGHTS_OTHER)
            # Further Specifications
            elif token_tag.endswith(FURTHER_SPECIFICATION):
                sample_labels.append(7)
                sample_sample_weights.append(WEIGHTS_OTHER)
            # Condition Specifications
            elif token_tag.endswith(CONDITION_SPECIFICATION):
                sample_labels.append(8)
                sample_sample_weights.append(WEIGHTS_OTHER)
    dataset_sample_weights.append(sample_sample_weights)            
    dataset_labels.append(sample_labels)
    dataset_word_ids.append(tokenization.word_ids())

dataset_labels = tf.constant(dataset_labels)
dataset_sample_weights = tf.constant(dataset_sample_weights)
dataset_tokens = dataset_tokens
print(dataset_labels.shape)
print(dataset_sample_weights.shape)
print(dataset_tokens['input_ids'].shape)

(417, 64)
(417, 64)
(417, 64)


In [222]:
dataset_labels

<tf.Tensor: shape=(417, 64), dtype=int32, numpy=
array([[0, 2, 8, ..., 0, 0, 0],
       [0, 6, 6, ..., 0, 0, 0],
       [0, 6, 6, ..., 0, 0, 0],
       ...,
       [0, 2, 8, ..., 0, 0, 0],
       [0, 5, 5, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])>

In [206]:
print("samples with xor:", len(debug_samples_with_xor), "; xor tokens", debug_xor_tokens)
print("samples with and:", len(debug_samples_with_and), "; and tokens", debug_and_tokens)

samples with xor: 108 ; xor tokens 117
samples with and: 8 ; and tokens 8


In [240]:
# split up
val_share = 1-0.1
val_instances = round(dataset_tokens['input_ids'].shape[0] * val_share)

def create_splitted_datasets(tokens, labels, sample_weights, split_index, batch_size=None):
    print(labels)
    tokens1, tokens2 = {k: v[:split_index] for k, v in tokens.items()}, {k: v[split_index:] for k, v in tokens.items()}
    labels1, labels2 = labels[:split_index], labels[split_index:]
    sample_weights1, sample_weights2 = sample_weights[:split_index], sample_weights[split_index:]
    
    create_dataset = lambda t, l, w: tf.data.Dataset.from_tensor_slices(({'input_ids': t['input_ids'], 'attention_mask': t['attention_mask']}, 
                                                                         l, w))
    train_dataset = create_dataset(tokens1, labels1, sample_weights1)
    dev_dataset = create_dataset(tokens2, labels2, sample_weights2)
    
    if batch_size:
        train_dataset = train_dataset.batch(batch_size)
        dev_dataset = dev_dataset.batch(batch_size)
    
    return train_dataset, dev_dataset

train_dataset, dev_dataset = create_splitted_datasets(dataset_tokens, dataset_labels, dataset_sample_weights, 
                                                      val_instances, batch_size=8)
print(f"{len(train_dataset)} / {len(dev_dataset)}")

tf.Tensor(
[[0 2 8 ... 0 0 0]
 [0 6 6 ... 0 0 0]
 [0 6 6 ... 0 0 0]
 ...
 [0 2 8 ... 0 0 0]
 [0 5 5 ... 0 0 0]
 [0 1 1 ... 0 0 0]], shape=(417, 64), dtype=int32)
47 / 6


In [241]:
for i, s in enumerate(dev_dataset):
    if i == 1:
        print(len(sample_sentences[val_instances + i]), sample_sentences[val_instances + i])
        t, l, w = s
        print(t)
        print(l)
        print(w)
        print()

16 ['The', 'process', 'is', 'triggered', 'by', 'the', 'demand', 'of', 'a', 'functional', 'department', 'to', 'fill', 'a', 'post', '.']
{'input_ids': <tf.Tensor: shape=(8, 64), dtype=int32, numpy=
array([[  101,  1996,  2175,  4618,  2025,  8757,  1996,  5796,  2361,
         2055,  1996,  5080,  3431,  1010,  1996,  3040,  2951,  1010,
         1996,  8316,  4175,  2012,  4487, 25855, 16671,  2075,  1010,
         1998,  1996,  8316,  4175,  2012,  8272,  1012,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [  101,  2016,  2059, 14148,  1996,  1999,  6767,  6610,  5167,
         1998,  3964,  1996,  7978,  3465,  2415,  2012,  1996,  9353,
         4168, 12943,  1998,  1996,  3141,  3465,  2415, 10489,  2005,
         2169,  2597,  2006,  1037,  3584,  2433,  1006,  1036,  1036,
       

In [225]:
print(dev_dataset)

<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(64,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(64,), dtype=tf.int32, name=None)}, TensorSpec(shape=(64,), dtype=tf.int32, name=None), TensorSpec(shape=(64,), dtype=tf.float32, name=None))>


## Create Model

In [229]:
# user num_labels=4 when classifying only in XOR, AND and OTHER; or num_labels=9 if in all classes
token_cls_model = transformers.TFAutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=9)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForTokenClassification: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferenc

In [230]:
type(token_cls_model)
token_cls_model.summary()

Model: "tf_distil_bert_for_token_classification_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 dropout_99 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  6921      
                                                                 
Total params: 66,369,801
Trainable params: 66,369,801
Non-trainable params: 0
_________________________________________________________________


### a) including Model in native Keras

In [242]:
import argparse
import os
import datetime
import re

parser = argparse.ArgumentParser()
# Standard params
parser.add_argument("--batch_size", default=8, type=int, help="Batch size.")
parser.add_argument("--epochs", default=1, type=int, help="Number of epochs.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
# Architecture params
parser.add_argument("--huggingface_model_name", default="distilbert-base-uncased", type=str, help="Model checkpoint")

args = parser.parse_args([] if "__file__" not in globals() else None)


class GatewayTokenClassifier(tf.keras.Model):

    def __init__(self, args: argparse.Namespace, model, train_dataset: tf.data.Dataset) -> None:

        # A) OPTIMIZER
        optimizer, lr_schedule = transformers.create_optimizer(
            init_lr=2e-5,
            num_train_steps=(len(train_dataset) // args.batch_size) * args.epochs,
            weight_decay_rate=0.01,
            num_warmup_steps=0,
        )

        # B) ARCHITECTURE
        inputs = {
            "input_ids": tf.keras.layers.Input(shape=[None], dtype=tf.int32),
            "attention_mask": tf.keras.layers.Input(shape=[None], dtype=tf.int32)
        }
        predictions = model(inputs)
        super().__init__(inputs=inputs, outputs=predictions)

        # C) COMPILE
        self.compile(optimizer=optimizer,
                     #loss=custom_loss,
                     loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                     weighted_metrics=[tf.metrics.SparseCategoricalAccuracy(name="accuracy"),
#                                                                tf.metrics.Precision(name="precision"),
#                                                                tf.metrics.Recall(name="recall"),
#                                                                tfa.metrics.F1Score(4, name="f1")
                              ])
        token_cls_model.summary()


# Create logdir name
args.logdir = os.path.join("data/logs", "{}-{}-{}".format(
    os.path.basename(globals().get("__file__", "notebook")),
    datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"),
    ",".join(("{}={}".format(re.sub("(.)[^_]*_?", r"\1", k), v) for k, v in sorted(vars(args).items())))
))


# Create the model and train it
model = GatewayTokenClassifier(args, token_cls_model, train_dataset)
model.fit(
    train_dataset, batch_size=args.batch_size, epochs=3, validation_data=dev_dataset,
    callbacks=[tf.keras.callbacks.TensorBoard(args.logdir, histogram_freq=1, update_freq=100, profile_batch=0)]
)

Model: "tf_distil_bert_for_token_classification_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 dropout_99 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  6921      
                                                                 
Total params: 66,369,801
Trainable params: 66,369,801
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
47/47 [==============================] - 110s 2s/step - loss: 0.2049 - accuracy: 0.4074 - val_loss: 0.2083 - val_accuracy: 0.4700
Epoch 2/3
47/47 [==============================] - 93s 2s/step - loss: 

In [243]:
# : Predict the tags on the test set.
print("create dev set predictions")
predictions = model.predict(dev_dataset)

create dev set predictions


In [244]:
import numpy as np

print(type(predictions))
logits = predictions.logits
print("Logits shape:", tf.shape(logits))
print()

for i, sample in enumerate(logits):
    print(f"sample {i}".center(50, '-'))
    sample_word_ids = all_word_ids[val_instances +i]
    print(sample_sentences[val_instances+i])
    important_token_indices = [i for i, word_id in enumerate(sample_word_ids) if word_id != None]
    for j, token in enumerate(sample):
        if j in important_token_indices:
            print(j, token, end=' ')
            token_label = np.argmax(token)
            print(token_label, end=' ')
            if token_label in [2,3]:
                print("!", end=' ')
            print(sample_sentences[val_instances+i][sample_word_ids[j]], end=' ')
            print()
    print()
    

<class 'transformers.modeling_tf_outputs.TFTokenClassifierOutput'>
Logits shape: tf.Tensor([42 64  9], shape=(3,), dtype=int32)

---------------------sample 0---------------------
['If', 'the', 'treasurer', 'accepts', 'the', 'expenses', 'for', 'processing', ',', 'the', 'report', 'moves', 'to', 'an', 'automatic', 'activity', 'that', 'links', 'to', 'a', 'payment', 'system', '.']
1 [ 0.1642295   0.32701483  0.32928362 -0.28635964  0.07606002  0.20461005
  0.09435598 -0.22771218 -0.04298494] 2 ! If 
2 [ 0.13069834  0.3357764  -0.01420432 -0.5717539  -0.06914806  0.24136728
  0.4111342  -0.41454947  0.1557002 ] 6 the 
3 [ 0.20721373  0.28678545 -0.17796335 -0.36890748  0.04325901  0.12306911
  0.35251665 -0.17536986  0.23240381] 6 treasurer 
4 [ 0.27884334  0.3738095  -0.0709691  -0.21859549 -0.11548982  0.38964114
  0.16397166 -0.3193674  -0.02246042] 5 accepts 
5 [ 0.30359808  0.62666273 -0.03764751 -0.32504827 -0.07952991  0.38368902
  0.29756245 -0.30660066  0.10364936] 1 the 
6 [ 0.428

In [200]:
# results of 4 label cls

<class 'transformers.modeling_tf_outputs.TFTokenClassifierOutput'>
Logits shape: tf.Tensor([42 64  4], shape=(3,), dtype=int32)

---------------------sample 0---------------------
['If', 'the', 'treasurer', 'accepts', 'the', 'expenses', 'for', 'processing', ',', 'the', 'report', 'moves', 'to', 'an', 'automatic', 'activity', 'that', 'links', 'to', 'a', 'payment', 'system', '.']
1 [-0.68042624 -0.06963277 -0.05562851 -0.45490682] 2 ! If 
2 [-0.67450744 -0.07063795 -0.07084563 -0.4472993 ] 1 the 
3 [-0.68007725 -0.06729331 -0.0786977  -0.443125  ] 1 treasurer 
4 [-0.66857904 -0.07492305 -0.05826456 -0.44343913] 2 ! accepts 
5 [-0.67852056 -0.08177662 -0.08233006 -0.43491125] 1 the 
6 [-0.68491924 -0.07899152 -0.06896686 -0.45671514] 2 ! expenses 
7 [-0.6832283  -0.07021421 -0.06331892 -0.4526383 ] 2 ! for 
8 [-0.6835841  -0.06967415 -0.06367798 -0.44890505] 2 ! processing 
9 [-0.67150086 -0.06618639 -0.07021592 -0.44545662] 1 , 
10 [-0.6836521  -0.06786406 -0.05845393 -0.44633073] 2 ! the

### b) with transformers Trainer (NOT WORKING WITH TF)

In [13]:
batch_size = 8
args = transformers.TrainingArguments(
    output_dir='./results',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01
)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

trainer = transformers.TFTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Users\janek\anaconda3\envs\thesis\lib\site-packages\transformers\trainer_tf.py:115: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/main/examples/tensorflow
  warnings.warn(


ImportError: Method `n_gpu` requires PyTorch.